<a href="https://colab.research.google.com/github/Maedeabm/Fraud-Detection-Using-Neural-Network/blob/main/Fraud_Detection_LSTM_PaySim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given the nature of the PaySim dataset (sequential and imbalanced) and the current advances in time series processing, I'd suggest using LSTM (Long Short-Term Memory) networks. They are designed to work with sequence data and can capture long-term dependencies which are crucial for such datasets.

Let's start with a basic implementation of an LSTM using TensorFlow and Keras in a Google Colab environment.

1. Setting Up Google Colab:

Open Google Colab and start a new notebook. Use the following initial setup:

In [ ]:
!pip install tensorflow

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

2. Load the PaySim Dataset:

PaySim:

  Description: PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country.
    
  Features: Step (time), type, amount, name of origin, old balance of origin, new balance of origin, name of destination, old balance of destination, new balance of destination, fraud (binary).
  
  Link: PaySim on Kaggle

First, upload the dataset to Colab:

You can access the PaySim dataset directly from Kaggle using their API. Here's how you can do that on Google Colab:
1. Setting Up Kaggle in Google Colab:

1.1 If you don’t have the Kaggle API token, you need to create it:

  Go to your Kaggle account settings.
    
  Click on 'Create New API Token'.
  
  This will download a file named kaggle.json.

1.2 Upload this to Google Colab:

In [ ]:
from google.colab import files

uploaded = files.upload()

# Assuming the dataset is named "paysim.csv"
data = pd.read_csv('paysim.csv')


Saving paysim.csv to paysim.csv


3. Data Preprocessing:

This will be a basic preprocessing to get started:

In [ ]:
# Dropping columns that may not be required for this basic model
data = data.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

# Convert categorical columns to numerical values
data = pd.get_dummies(data, columns=['type'], drop_first=True)

# Normalize the features
scaler = MinMaxScaler()
data[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']] = scaler.fit_transform(data[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']])

# Splitting data into features and target variable
X = data.drop('isFraud', axis=1).values
y = data['isFraud'].values

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input to be 3D for LSTM [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


4. Define and Train the LSTM Model:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping


model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Implementing early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=2, callbacks=[es])


Epoch 1/100
63627/63627 - 261s - loss: 0.0096 - accuracy: 0.9987 - val_loss: 0.0104 - val_accuracy: 0.9987 - 261s/epoch - 4ms/step
Epoch 2/100
63627/63627 - 252s - loss: 0.0091 - accuracy: 0.9987 - val_loss: 0.0082 - val_accuracy: 0.9987 - 252s/epoch - 4ms/step
Epoch 3/100
63627/63627 - 253s - loss: 0.0089 - accuracy: 0.9987 - val_loss: 0.0081 - val_accuracy: 0.9987 - 253s/epoch - 4ms/step
Epoch 4/100
63627/63627 - 237s - loss: 0.0088 - accuracy: 0.9987 - val_loss: 0.0082 - val_accuracy: 0.9987 - 237s/epoch - 4ms/step
Epoch 5/100
63627/63627 - 252s - loss: 0.0086 - accuracy: 0.9987 - val_loss: 0.0077 - val_accuracy: 0.9987 - 252s/epoch - 4ms/step
Epoch 6/100
63627/63627 - 251s - loss: 0.0085 - accuracy: 0.9987 - val_loss: 0.0077 - val_accuracy: 0.9987 - 251s/epoch - 4ms/step
Epoch 7/100
63627/63627 - 235s - loss: 0.0084 - accuracy: 0.9987 - val_loss: 0.0075 - val_accuracy: 0.9987 - 235s/epoch - 4ms/step
Epoch 8/100
63627/63627 - 249s - loss: 0.0082 - accuracy: 0.9987 - val_loss: 0.0078

5. Model Evaluation:

In [ ]:
y_prob = model.predict(X_test)
y_pred = (y_prob > 0.5).astype(int).flatten()

print(classification_report(y_test, y_pred))



39767/39767 [==============================] - 65s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.99      0.22      0.35      1620

    accuracy                           1.00   1272524
   macro avg       1.00      0.61      0.68   1272524
weighted avg       1.00      1.00      1.00   1272524



The result you provided is a classification report, commonly used for evaluating the performance of a classification model. Let's break down what the metrics suggest:

  Class 0 (Not Fraudulent Transactions):
        Precision: The model is 100% precise when it predicts a transaction as non-fraudulent, which means every time it said a transaction was legit, it was right.
        Recall: The model is also capturing 100% of the non-fraudulent transactions.

  Class 1 (Fraudulent Transactions):
        Precision: The precision is 99%, meaning that when the model predicts a transaction is fraudulent, it's correct 99% of the time.
        Recall: The recall is 22%. This means that the model only captures 22% of all the actual fraudulent transactions. This is concerning because 78% of fraudulent transactions are going undetected.
        F1-score: This is a harmonic mean of precision and recall. Given that the recall is low, the F1-score for the fraudulent class is also quite low at 35%.

  Accuracy: While the overall accuracy is 100%, accuracy can be misleading in imbalanced datasets (like this one, where fraudulent transactions are much fewer). A model could predict every transaction as non-fraudulent and still achieve a very high accuracy due to the class imbalance.

  Macro Avg: This averages the unweighted mean per label. The average precision and recall are both high, but the F1-score (which considers both) is relatively low due to the low recall for the fraudulent class.

  Weighted Avg: This averages the support-weighted mean per label. These numbers are very high because the vast majority of the dataset consists of the non-fraudulent class.

Is it a good result?
While the results for the non-fraudulent class are excellent, the primary concern is the low recall for the fraudulent class. In the context of fraud detection, false negatives (fraudulent transactions that the model fails to detect) can be very costly. Even if the model is precise in its predictions of fraud, it's missing a significant portion of them.

To improve this, you might want to:

  Address the class imbalance further, possibly with techniques like SMOTE or ADASYN.
  
  Experiment with different models or architectures.
  
  Fine-tune the model, focusing on improving recall for the fraudulent class, possibly by adjusting the decision threshold.
  
  Consider ensemble methods or more advanced techniques.

In fraud detection, maximizing recall for the fraudulent class (while keeping precision reasonably high) is often a primary objective.